# 本文件实现 AES-128/192/256 加密算法，并完成 CBC 和 CTR 模式
### （<span style='color:red'>注意，本文件不支持对明文的填充</span>）
- 请启动jupyter python服务，若启动jupyter sagemath服务，请修改异或符号

## 首先进行 `S盒` 与 `S盒的逆盒` 生成

In [1]:
def mybin(target: int, bit=8):
    """把整数转化为二进制字符串，长度为bit位，不足的前面补0"""
    target = bin(target)[2:]
    return "0" * (bit - len(target)) + target if len(target) < bit else target[-bit:]

def poly_div(a: int, b: int):
    """多项式除法"""
    if b == 0:
        raise ValueError("除数不能为0")
    divisor = 0
    aidx = mybin(a, bit=9).index("1")
    bidx = mybin(b, bit=9).index("1") 
    while (aidx <= bidx): 
        divisor += 2 ** (bidx - aidx)
        a ^= b << (bidx - aidx)
        if a == 0: break
        aidx = mybin(a, bit=9).index("1")
    return divisor, a

def poly_mul(a: int, b: int, m: int):
    """GF(2)域上的多项式乘法"""
    m = m - 128 if m > 127 else m
    bstr = mybin(b)
    ans = 0
    if bstr[-1] == "1": ans ^= a  
    for i in range(1, 8):  
        if a > 127:  
            a <<= 1
            a = (a - 128) ^ m  
        else:
            a <<= 1 
        if bstr[8 - i - 1] == "1": 
            ans ^= a
    return ans

def poly_gcd(a: int, b: int, m: int):
    """计算多项式的最大公约数"""
    r1, r2 = sorted([a, b], reverse=True)
    w1, w2 = 0, 1
    while int(r2):
        divisor = poly_div(r1, r2)
        r1, r2 = r2, divisor[1]
        w1, w2 = w2, w1 ^ poly_mul(divisor[0], w2, m)
    return r1, w1

translis = [str(i) for i in range(10)] + [chr(ord("A") + i) for i in range(6)]
inv_transdic = dict(zip(translis, list(range(16))))

def sbox_to_standard(sbox_2d):
    """将二维字符串列表形式的S盒转换为标准AES S盒的一维整数列表格式"""
    return [byte for row in sbox_2d for byte in row]

def transform(a: int) -> str:
    """将八位的二进制数转换为十六进制字符串"""
    return translis[a // 16] + translis[a % 16]

def inv_transform(b: str) -> int:
    """返回十六进制字符串对应的十进制整数"""
    return (inv_transdic[b[0]] << 4) + inv_transdic[b[1]]

def SBox(m : int = 283, C : str = '63'):
    """生成S盒"""
    C = [int(i) for i in list(mybin(inv_transform(C)))][::-1]
    Sbox = [[poly_gcd(r * 16 + c, m, 0)[1] for c in range(16)] for r in range(16)]
    for r in range(16):
        for c in range(16):
            b = [int(i) for i in list(mybin(Sbox[r][c]))][::-1]
            Sbox[r][c] = 0
            for i in range(8):
                Sbox[r][c] += (2 ** i) * (
                            b[i] ^ b[(i + 4) % 8] ^ b[(i + 5) % 8] ^ b[(i + 6) % 8] ^ b[(i + 7) % 8] ^ C[i])
    return Sbox

def SBox_inv(sbox: list):
    """生成S盒的逆盒"""
    I_sbox = [[0 for col in range(16)] for row in range(16)]
    for i in range(16):
        for j in range(16):
            m = sbox[i][j]
            a = (int)(m / 16)
            b = m % 16
            I_sbox[a][b] = i * 16 + j
    return I_sbox

# ----------下面这两个盒仅用于校验，并未用于其他过程---------- #
std_Sbox = [
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
]

std_InvSbox = [
    0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
    0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
    0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
    0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
    0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
    0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
    0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
    0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
    0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
    0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
    0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
    0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
    0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
    0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
    0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d
]
# ----------上面这两个盒仅用于校验，并未用于其他过程---------- #
def check_Box(sbox, inv_box):
    """检查S盒和逆S盒是否正确"""
    for item in sbox:
        if item < 0 or item > 255:
            raise ValueError("S盒中的值必须在0到255之间")
    for item in inv_box:
        if item < 0 or item > 255:
            raise ValueError("逆S盒中的值必须在0到255之间")
    for test, standard in zip(sbox, std_Sbox):
        if test != standard:
            raise ValueError("S盒不正确")
    for test, standard in zip(inv_box, std_InvSbox):
        if test != standard:
            raise ValueError("逆S盒不正确")
    print("S盒和逆S盒均正确")

def show(sbox):
    """打印一维整数列表形式的S盒"""
    for i in range(16):
        row = sbox[i*16 : (i+1)*16]
        print(" ".join(f"{byte:02x}" for byte in row))
    print()

In [2]:
Sbox = SBox()
InvSbox = SBox_inv(Sbox)

Sbox = sbox_to_standard(Sbox)
InvSbox = sbox_to_standard(InvSbox)

check_Box(Sbox, InvSbox)

# show(Sbox)
# show(InvSbox)

S盒和逆S盒均正确


### 这里构造一个函数 SBOX he INVSBOX，接受一个字节，返回一个字节

In [ ]:
import galois
def SBOX(a: int) -> int:
    """S盒变换"""
    GF = galois.GF(2**8, irreducible_poly=0x11B)
    b = int(GF(a) ** (-1)) if a != 0 else 0
    return b ^ ((b << 1) & 0xff | (b >> 7)) ^ ((b << 2) & 0xff | (b >> 6)) ^ ((b << 3) & 0xff | (b >> 5)) ^ ((b << 4) & 0xff | (b >> 4)) ^ 0x63

def INVSBOX(c : int) -> int:
    """S盒的逆变换"""
    b = ((c << 1) & 0xff | (c >> 7)) ^ ((c << 3) & 0xff | (c >> 5)) ^ ((c << 6) & 0xff | (c >> 2)) ^ 0x05
    # 在GF(2^8)上求逆
    GF = galois.GF(2**8, irreducible_poly=0x11B)
    return int(GF(b) ** (-1)) if b != 0 else 0

# 实际上，可以不使用galois库：
'''
def SBOX(a:int) -> int:
    """S盒变换"""
    if a == 0:
        return 0x63  
    u, v = a, 0x11B  
    x1, x2 = 1, 0
    while u != 1:
        j = len(bin(u)) - len(bin(v))
        if j < 0:
            u, v = v, u
            x1, x2 = x2, x1
            continue
        u ^= (v << j)
        x1 ^= (x2 << j)
    b = x1
    c = b
    for i in range(4):  
        b = ((b << 1) | (b >> 7)) & 0xFF
        c ^= b
    return c ^ 0x63

def INVSBOX(c:int) -> int:
    """S盒的逆变换"""
    b = 0
    for i in range(8):
        bit1 = (c >> ((i + 7) % 8)) & 1  
        bit3 = (c >> ((i + 5) % 8)) & 1  
        bit6 = (c >> ((i + 2) % 8)) & 1  
        new_bit = (bit1 ^ bit3 ^ bit6 ^ ((0x05 >> i) & 1))
        b |= (new_bit << i)
    if b == 0:
        return 0
    poly = 0x11B 
    u, v = b, poly
    x1, x2 = 1, 0
    while u != 1:
        j = u.bit_length() - v.bit_length()
        if j < 0:
            u, v = v, u
            x1, x2 = x2, x1
            j = -j
        u ^= (v << j)
        x1 ^= (x2 << j)
    a = x1
    return a
'''


## 然后复制密钥派生函数的代码

In [4]:
Rcon = [0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]

def key_expansion(key_bytes, key_size_in_bits):
    Nk = key_size_in_bits // 32
    if Nk == 4: Nr = 10
    elif Nk == 6: Nr = 12
    elif Nk == 8: Nr = 14
    Nb = 4
    w = [[0]*4 for _ in range(Nb * (Nr + 1))]
    for i in range(Nk):
        w[i] = list(key_bytes[4*i : 4*(i+1)])
    for i in range(Nk, Nb * (Nr + 1)):
        temp = list(w[i-1])
        if i % Nk == 0:
            temp = temp[1:] + temp[:1]
            temp = [SBOX(b) for b in temp]
            temp[0] = temp[0] ^ Rcon[i // Nk]
        elif Nk > 6 and i % Nk == 4:
            temp = [SBOX(b) for b in temp]
        w[i] = [w[i-Nk][j] ^ temp[j] for j in range(4)]
    round_keys_matrices = []
    for round_num in range(Nr + 1):
        round_key_matrix = [[0]*4 for _ in range(4)]
        for c in range(4):
            word_from_w = w[round_num * Nb + c]
            for r in range(4):
                round_key_matrix[r][c] = word_from_w[r]
        round_keys_matrices.append(round_key_matrix)
    return round_keys_matrices

## 接着进行 AES block加密/解密

In [5]:
def hex_to_bytes(hex_str):
    return bytes.fromhex(hex_str)

def bytes_to_hex(bytes_data):
    if isinstance(bytes_data, list):
        bytes_data = bytes(bytes_data)
    return bytes_data.hex().upper()

def bytes_to_state(bytes_data):
    state = [[0]*4 for _ in range(4)]
    for i in range(16):
        state[i % 4][i // 4] = bytes_data[i]
    return state

def state_to_bytes(state):
    bytes_data = []
    for i in range(4):
        for j in range(4):
            bytes_data.append(state[j][i])
    return bytes_data

def add_round_key(state, round_key):
    for i in range(4):
        for j in range(4):
            state[i][j] ^= round_key[i][j]
    return state

def sub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = SBOX(state[i][j])
    return state

def shift_rows(state):
    state[1] = state[1][1:] + state[1][:1]
    state[2] = state[2][2:] + state[2][:2]
    state[3] = state[3][3:] + state[3][:3]
    return state

def mix_columns(state):
    for i in range(4):
        a0 = state[0][i]
        a1 = state[1][i]
        a2 = state[2][i]
        a3 = state[3][i]
        
        state[0][i] = gf_mul(0x02, a0) ^ gf_mul(0x03, a1) ^ a2 ^ a3
        state[1][i] = a0 ^ gf_mul(0x02, a1) ^ gf_mul(0x03, a2) ^ a3
        state[2][i] = a0 ^ a1 ^ gf_mul(0x02, a2) ^ gf_mul(0x03, a3)
        state[3][i] = gf_mul(0x03, a0) ^ a1 ^ a2 ^ gf_mul(0x02, a3)
    return state

def gf_mul(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        high_bit_set = a & 0x80
        a <<= 1
        if high_bit_set:
            a ^= 0x1B  # x^8 + x^4 + x^3 + x + 1
        b >>= 1
    return p & 0xFF

def inv_sub_bytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = INVSBOX(state[i][j])
    return state

def inv_shift_rows(state):
    state[1] = state[1][-1:] + state[1][:-1]
    state[2] = state[2][-2:] + state[2][:-2]
    state[3] = state[3][-3:] + state[3][:-3]
    return state

def inv_mix_columns(state):
    for i in range(4):
        a0 = state[0][i]
        a1 = state[1][i]
        a2 = state[2][i]
        a3 = state[3][i]
        
        state[0][i] = gf_mul(0x0E, a0) ^ gf_mul(0x0B, a1) ^ gf_mul(0x0D, a2) ^ gf_mul(0x09, a3)
        state[1][i] = gf_mul(0x09, a0) ^ gf_mul(0x0E, a1) ^ gf_mul(0x0B, a2) ^ gf_mul(0x0D, a3)
        state[2][i] = gf_mul(0x0D, a0) ^ gf_mul(0x09, a1) ^ gf_mul(0x0E, a2) ^ gf_mul(0x0B, a3)
        state[3][i] = gf_mul(0x0B, a0) ^ gf_mul(0x0D, a1) ^ gf_mul(0x09, a2) ^ gf_mul(0x0E, a3)
    return state

def aes_encrypt_block(plaintext_block, key_bytes, key_size_in_bits):
    round_keys = key_expansion(key_bytes, key_size_in_bits)
    state = bytes_to_state(plaintext_block)
    state = add_round_key(state, round_keys[0])
    
    Nk = key_size_in_bits // 32
    if Nk == 4: Nr = 10
    elif Nk == 6: Nr = 12
    elif Nk == 8: Nr = 14
    
    for round_num in range(1, Nr):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, round_keys[round_num])
    
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[Nr])
    
    return state_to_bytes(state)

def aes_decrypt_block(ciphertext_block, key_bytes, key_size_in_bits):
    round_keys = key_expansion(key_bytes, key_size_in_bits)
    state = bytes_to_state(ciphertext_block)
    state = add_round_key(state, round_keys[-1])
    
    Nk = key_size_in_bits // 32
    if Nk == 4: Nr = 10
    elif Nk == 6: Nr = 12
    elif Nk == 8: Nr = 14
    
    for round_num in range(Nr-1, 0, -1):
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
        state = add_round_key(state, round_keys[round_num])
        state = inv_mix_columns(state)
    
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    state = add_round_key(state, round_keys[0])
    
    return state_to_bytes(state)

## 然后进行 AES 加密/解密

In [6]:
# 实际上，AES加密与ECB模式一样
def aes_encrypt(plaintext, key, key_size_in_bits):
    """AES加密函数"""
    block_size = 16
    encrypted_data = b''
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        encrypted_block = aes_encrypt_block(block, key, key_size_in_bits)
        if isinstance(encrypted_block, list):
            encrypted_block = bytes(encrypted_block)
        encrypted_data += encrypted_block
    return encrypted_data

def aes_decrypt(ciphertext, key, key_size_in_bits):
    """AES解密函数"""
    block_size = 16
    decrypted_data = b''
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        decrypted_block = aes_decrypt_block(block, key, key_size_in_bits)
        if isinstance(decrypted_block, list):
            decrypted_block = bytes(decrypted_block)
        decrypted_data += decrypted_block
    return decrypted_data

## 接着进行 AES 工作模式的加密/解密

In [7]:
def cbc_encrypt(plaintext, key_bytes, iv_bytes, use_padding=True):
    block_size = 16
    if use_padding:
        # 填充明文到块大小的整数倍 (PKCS#7 填充)
        padding_length = block_size - (len(plaintext) % block_size)
        padded_plaintext = plaintext + bytes([padding_length]) * padding_length
    else:
        if len(plaintext) % block_size != 0:
            raise ValueError("明文长度必须是块大小的整数倍，或者启用填充")
        padded_plaintext = plaintext
    
    ciphertext = bytearray()
    previous_block = iv_bytes
    
    for i in range(0, len(padded_plaintext), block_size):
        block = padded_plaintext[i:i+block_size]
        # 异或上一个密文块 (或 IV)
        xored_block = bytes(a ^ b for a, b in zip(block, previous_block))
        encrypted_block = aes_encrypt_block(xored_block, key_bytes, len(key_bytes) * 8)
        ciphertext.extend(encrypted_block)
        previous_block = encrypted_block
    
    return bytes(ciphertext)

def cbc_decrypt(ciphertext, key_bytes, iv_bytes, use_padding=True):
    block_size = 16
    if len(ciphertext) % block_size != 0:
        raise ValueError("密文长度必须是块大小的整数倍")
    
    plaintext = bytearray()
    previous_block = iv_bytes
    
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]
        decrypted_block = aes_decrypt_block(block, key_bytes, len(key_bytes) * 8)
        # 异或上一个密文块 (或 IV)
        plaintext_block = bytes(a ^ b for a, b in zip(decrypted_block, previous_block))
        plaintext.extend(plaintext_block)
        previous_block = block
    
    # 移除填充 (PKCS#7)
    if use_padding and plaintext:
        padding_length = plaintext[-1]
        if padding_length < 1 or padding_length > block_size:
            raise ValueError("无效的填充长度")
        if plaintext[-padding_length:] != bytes([padding_length]) * padding_length:
            raise ValueError("无效的填充")
        return bytes(plaintext[:-padding_length])
    return plaintext

def increment_counter(counter_bytes):
    """递增CTR模式的计数器值（大端序）"""
    counter = int.from_bytes(counter_bytes, 'big')
    counter += 1
    return counter.to_bytes(16, 'big')

def ctr_encrypt(plaintext, key_bytes, init_counter_bytes):
    """CTR模式加密（与解密逻辑相同）"""
    block_size = 16
    ciphertext = bytearray()
    counter = init_counter_bytes
    
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]
        
        # 加密当前计数器值
        keystream_block = aes_encrypt_block(counter, key_bytes, len(key_bytes) * 8)
        
        # 与明文块异或（处理最后一个可能不足块大小的块）
        xored_block = bytes(a ^ b for a, b in zip(block, keystream_block[:len(block)]))
        ciphertext.extend(xored_block)
        
        # 递增计数器
        counter = increment_counter(counter)
    
    return bytes(ciphertext)

def ctr_decrypt(ciphertext, key_bytes, init_counter_bytes):
    return ctr_encrypt(ciphertext, key_bytes, init_counter_bytes)

## 最后进行测试

In [8]:
print("\n=== AES-128 模式测试 ===")
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "3AD77BB40D7A3660A89ECAF32466EF97F5D3D58503B9699DE785895A96FDBAAF43B1CD7F598ECE23881B00E3ED0306887B0C785E27E8AD3F8223207104725DD4"
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key)

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = aes_encrypt(plaintext_bytes, key_bytes, 128)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = aes_decrypt(hex_to_bytes(Ciphertext), key_bytes, 128)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-128 模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: 3AD77BB40D7A3660A89ECAF32466EF97F5D3D58503B9699DE785895A96FDBAAF43B1CD7F598ECE23881B00E3ED0306887B0C785E27E8AD3F8223207104725DD4
预期结果: 3AD77BB40D7A3660A89ECAF32466EF97F5D3D58503B9699DE785895A96FDBAAF43B1CD7F598ECE23881B00E3ED0306887B0C785E27E8AD3F8223207104725DD4
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [9]:
print("\n=== AES-192 模式测试 ===")
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "BD334F1D6E45F25FF712A214571FA5CC974104846D0AD3AD7734ECB3ECEE4EEFEF7AFD2270E2E60ADCE0BA2FACE6444E9A4B41BA738D6C72FB16691603C18E0E" 
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key)

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = aes_encrypt(plaintext_bytes, key_bytes, 192)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = aes_decrypt(hex_to_bytes(Ciphertext), key_bytes, 192)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-192 模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: BD334F1D6E45F25FF712A214571FA5CC974104846D0AD3AD7734ECB3ECEE4EEFEF7AFD2270E2E60ADCE0BA2FACE6444E9A4B41BA738D6C72FB16691603C18E0E
预期结果: BD334F1D6E45F25FF712A214571FA5CC974104846D0AD3AD7734ECB3ECEE4EEFEF7AFD2270E2E60ADCE0BA2FACE6444E9A4B41BA738D6C72FB16691603C18E0E
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [10]:
print("\n=== AES-256 模式测试 ===")
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = "F3EED1BDB5D2A03C064B5A7E3DB181F8591CCB10D410ED26DC5BA74A31362870B6ED21B99CA6F4F9F153E7B1BEAFED1D23304B7A39F9F3FF067D8D8F9E24ECC7"
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key)

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = aes_encrypt(plaintext_bytes, key_bytes, 256)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = aes_decrypt(hex_to_bytes(Ciphertext), key_bytes, 256)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-256 模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: F3EED1BDB5D2A03C064B5A7E3DB181F8591CCB10D410ED26DC5BA74A31362870B6ED21B99CA6F4F9F153E7B1BEAFED1D23304B7A39F9F3FF067D8D8F9E24ECC7
预期结果: F3EED1BDB5D2A03C064B5A7E3DB181F8591CCB10D410ED26DC5BA74A31362870B6ED21B99CA6F4F9F153E7B1BEAFED1D23304B7A39F9F3FF067D8D8F9E24ECC7
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [11]:
print("\n=== AES-128 CBC模式测试 ===")
IV = "000102030405060708090A0B0C0D0E0F"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "7649ABAC8119B246CEE98E9B12E9197D5086CB9B507219EE95DB113A917678B273BED6B8E3C1743B7116E69E222295163FF1CAA1681FAC09120ECA307586E1A7"

iv_bytes = hex_to_bytes(IV)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key[:32]) 

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = cbc_encrypt(plaintext_bytes, key_bytes, iv_bytes, use_padding=False)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = cbc_decrypt(hex_to_bytes(Ciphertext), key_bytes, iv_bytes, use_padding=False)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-128 CBC模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: 7649ABAC8119B246CEE98E9B12E9197D5086CB9B507219EE95DB113A917678B273BED6B8E3C1743B7116E69E222295163FF1CAA1681FAC09120ECA307586E1A7
预期结果: 7649ABAC8119B246CEE98E9B12E9197D5086CB9B507219EE95DB113A917678B273BED6B8E3C1743B7116E69E222295163FF1CAA1681FAC09120ECA307586E1A7
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [12]:
print("\n=== AES-192 CBC模式测试 ===")
IV = "000102030405060708090A0B0C0D0E0F"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "4F021DB243BC633D7178183A9FA071E8B4D9ADA9AD7DEDF4E5E738763F69145A571B242012FB7AE07FA9BAAC3DF102E008B0E27988598881D920A9E64F5615CD"

iv_bytes = hex_to_bytes(IV)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key[:48]) 

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = cbc_encrypt(plaintext_bytes, key_bytes, iv_bytes, use_padding=False)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = cbc_decrypt(hex_to_bytes(Ciphertext), key_bytes, iv_bytes, use_padding=False)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-192 CBC模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: 4F021DB243BC633D7178183A9FA071E8B4D9ADA9AD7DEDF4E5E738763F69145A571B242012FB7AE07FA9BAAC3DF102E008B0E27988598881D920A9E64F5615CD
预期结果: 4F021DB243BC633D7178183A9FA071E8B4D9ADA9AD7DEDF4E5E738763F69145A571B242012FB7AE07FA9BAAC3DF102E008B0E27988598881D920A9E64F5615CD
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [13]:
print("\n=== AES-256 CBC模式测试 ===")
IV = "000102030405060708090A0B0C0D0E0F"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = "F58C4C04D6E5F1BA779EABFB5F7BFBD69CFC4E967EDB808D679F777BC6702C7D39F23369A9D9BACFA530E26304231461B2EB05E2C39BE9FCDA6C19078C6A9D1B"

iv_bytes = hex_to_bytes(IV)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key[:64]) 

if len(plaintext_bytes) % 16 == 0:
    print("注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试")

encrypted = cbc_encrypt(plaintext_bytes, key_bytes, iv_bytes, use_padding=False)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = cbc_decrypt(hex_to_bytes(Ciphertext), key_bytes, iv_bytes, use_padding=False)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-256 CBC模式测试 ===
注意: 测试向量的明文长度是块大小的整数倍，将使用不填充模式进行测试
加密结果: F58C4C04D6E5F1BA779EABFB5F7BFBD69CFC4E967EDB808D679F777BC6702C7D39F23369A9D9BACFA530E26304231461B2EB05E2C39BE9FCDA6C19078C6A9D1B
预期结果: F58C4C04D6E5F1BA779EABFB5F7BFBD69CFC4E967EDB808D679F777BC6702C7D39F23369A9D9BACFA530E26304231461B2EB05E2C39BE9FCDA6C19078C6A9D1B
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [14]:
print("\n=== AES-128 CTR模式测试 ===")
InitCounter = "F0F1F2F3F4F5F6F7F8F9FAFBFCFDFEFF"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "874D6191B620E3261BEF6864990DB6CE9806F66B7970FDFF8617187BB9FFFDFF5AE4DF3EDBD5D35E5B4F09020DB03EAB1E031DDA2FBE03D1792170A0F3009CEE"

init_counter_bytes = hex_to_bytes(InitCounter)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key)  

encrypted = ctr_encrypt(plaintext_bytes, key_bytes, init_counter_bytes)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = ctr_decrypt(hex_to_bytes(Ciphertext), key_bytes, init_counter_bytes)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-128 CTR模式测试 ===
加密结果: 874D6191B620E3261BEF6864990DB6CE9806F66B7970FDFF8617187BB9FFFDFF5AE4DF3EDBD5D35E5B4F09020DB03EAB1E031DDA2FBE03D1792170A0F3009CEE
预期结果: 874D6191B620E3261BEF6864990DB6CE9806F66B7970FDFF8617187BB9FFFDFF5AE4DF3EDBD5D35E5B4F09020DB03EAB1E031DDA2FBE03D1792170A0F3009CEE
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [15]:
print("\n=== AES-192 CTR模式测试 ===")
InitCounter = "F0F1F2F3F4F5F6F7F8F9FAFBFCFDFEFF"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "1ABC932417521CA24F2B0459FE7E6E0B090339EC0AA6FAEFD5CCC2C6F4CE8E941E36B26BD1EBC670D1BD1D665620ABF74F78A7F6D29809585A97DAEC58C6B050"

init_counter_bytes = hex_to_bytes(InitCounter)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key) 

encrypted = ctr_encrypt(plaintext_bytes, key_bytes, init_counter_bytes)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = ctr_decrypt(hex_to_bytes(Ciphertext), key_bytes, init_counter_bytes)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-192 CTR模式测试 ===
加密结果: 1ABC932417521CA24F2B0459FE7E6E0B090339EC0AA6FAEFD5CCC2C6F4CE8E941E36B26BD1EBC670D1BD1D665620ABF74F78A7F6D29809585A97DAEC58C6B050
预期结果: 1ABC932417521CA24F2B0459FE7E6E0B090339EC0AA6FAEFD5CCC2C6F4CE8E941E36B26BD1EBC670D1BD1D665620ABF74F78A7F6D29809585A97DAEC58C6B050
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


In [16]:
print("\n=== AES-256 CTR模式测试 ===")
InitCounter = "F0F1F2F3F4F5F6F7F8F9FAFBFCFDFEFF"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = "601EC313775789A5B7A7F504BBF3D228F443E3CA4D62B59ACA84E990CACAF5C52B0930DAA23DE94CE87017BA2D84988DDFC9C58DB67AADA613C2DD08457941A6"

init_counter_bytes = hex_to_bytes(InitCounter)
plaintext_bytes = hex_to_bytes(Plaintext)
key_bytes = hex_to_bytes(Key) 

encrypted = ctr_encrypt(plaintext_bytes, key_bytes, init_counter_bytes)
print("加密结果:", bytes_to_hex(encrypted))
print("预期结果:", Ciphertext)
print("加密测试通过:", bytes_to_hex(encrypted) == Ciphertext)

decrypted = ctr_decrypt(hex_to_bytes(Ciphertext), key_bytes, init_counter_bytes)
print("解密结果:", bytes_to_hex(decrypted))
print("原始明文:", Plaintext)
print("解密测试通过:", bytes_to_hex(decrypted) == Plaintext)


=== AES-256 CTR模式测试 ===
加密结果: 601EC313775789A5B7A7F504BBF3D228F443E3CA4D62B59ACA84E990CACAF5C52B0930DAA23DE94CE87017BA2D84988DDFC9C58DB67AADA613C2DD08457941A6
预期结果: 601EC313775789A5B7A7F504BBF3D228F443E3CA4D62B59ACA84E990CACAF5C52B0930DAA23DE94CE87017BA2D84988DDFC9C58DB67AADA613C2DD08457941A6
加密测试通过: True
解密结果: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
原始明文: 6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710
解密测试通过: True


## 以上均是从[这里](https://csrc.nist.gov/projects/cryptographic-standards-and-guidelines/example-values)获取的测试样例，都是不需要进行填充的

### AES的测试样例参照：
```python
# 值得一提的是，ECB模式起始和普通AES是一样的
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"

print("\n=== AES-128 模式测试 ===")
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "3AD77BB40D7A3660A89ECAF32466EF97F5D3D58503B9699DE785895A96FDBAAF43B1CD7F598ECE23881B00E3ED0306887B0C785E27E8AD3F8223207104725DD4"

print("\n=== AES-192 模式测试 ===")
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "BD334F1D6E45F25FF712A214571FA5CC974104846D0AD3AD7734ECB3ECEE4EEFEF7AFD2270E2E60ADCE0BA2FACE6444E9A4B41BA738D6C72FB16691603C18E0E" 

print("\n=== AES-256 模式测试 ===")
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = " F3EED1BDB5D2A03C064B5A7E3DB181F8591CCB10D410ED26DC5BA74A31362870B6ED21B99CA6F4F9F153E7B1BEAFED1D23304B7A39F9F3FF067D8D8F9E24ECC7"
```

### CBC模式的测试样例参照：
```python
IV = "000102030405060708090A0B0C0D0E0F"
Plaintext =  "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"

print("\n=== AES-128 CBC模式测试 ===")
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "7649ABAC8119B246CEE98E9B12E9197D5086CB9B507219EE95DB113A917678B273BED6B8E3C1743B7116E69E222295163FF1CAA1681FAC09120ECA307586E1A7"

print("\n=== AES-192 CBC模式测试 ===")
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "4F021DB243BC633D7178183A9FA071E8B4D9ADA9AD7DEDF4E5E738763F69145A571B242012FB7AE07FA9BAAC3DF102E008B0E27988598881D920A9E64F5615CD"

print("\n=== AES-256 CBC模式测试 ===")
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = "F58C4C04D6E5F1BA779EABFB5F7BFBD69CFC4E967EDB808D679F777BC6702C7D39F23369A9D9BACFA530E26304231461B2EB05E2C39BE9FCDA6C19078C6A9D1B"
```
### CTR模式的测试样例参照：
```python
InitCounter = "F0F1F2F3F4F5F6F7F8F9FAFBFCFDFEFF"
Plaintext = "6BC1BEE22E409F96E93D7E117393172AAE2D8A571E03AC9C9EB76FAC45AF8E5130C81C46A35CE411E5FBC1191A0A52EFF69F2445DF4F9B17AD2B417BE66C3710"

print("\n=== AES-128 CTR模式测试 ===")
Key = "2B7E151628AED2A6ABF7158809CF4F3C"
Ciphertext = "874D6191B620E3261BEF6864990DB6CE9806F66B7970FDFF8617187BB9FFFDFF5AE4DF3EDBD5D35E5B4F09020DB03EAB1E031DDA2FBE03D1792170A0F3009CEE"

print("\n=== AES-192 CTR模式测试 ===")
Key = "8E73B0F7DA0E6452C810F32B809079E562F8EAD2522C6B7B"
Ciphertext = "1ABC932417521CA24F2B0459FE7E6E0B090339EC0AA6FAEFD5CCC2C6F4CE8E941E36B26BD1EBC670D1BD1D665620ABF74F78A7F6D29809585A97DAEC58C6B050"

print("\n=== AES-256 CTR模式测试 ===")
Key = "603DEB1015CA71BE2B73AEF0857D77811F352C073B6108D72D9810A30914DFF4"
Ciphertext = "601EC313775789A5B7A7F504BBF3D228F443E3CA4D62B59ACA84E990CACAF5C52B0930DAA23DE94CE87017BA2D84988DDFC9C58DB67AADA613C2DD08457941A6"
```